In [10]:
import numpy as np
from header import *
from nsp.utils.base_conv import *
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *

In [11]:
Sz = np.zeros([3,3])
Sz[0,0] = 1
Sz[2,2] = -1
Sx = np.zeros([3, 3])
Sx[1,0] = Sx[0,1] = Sx[2,1] = Sx[1,2] = 1/np.sqrt(2)
Sy = np.zeros([3, 3], dtype=np.complex64)
Sy[1,0] = Sy[2,1] = 1j/np.sqrt(2)
Sy[0,1] = Sy[1,2] = -1j/np.sqrt(2)


SzSz = np.kron(Sz,Sz).astype(np.float64)
SxSx = np.kron(Sx,Sx).astype(np.float64)
SySy = np.kron(Sy,Sy).astype(np.float64)

lh = SzSz + SxSx + SySy
J = [1, 1/3]
lh = lh + (lh@lh - np.eye(9))*J[1]
lh = -lh

/tmp/ipykernel_3170/2890540848.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  SySy = np.kron(Sy,Sy).astype(np.float64)


In [12]:
np.set_printoptions(precision=3)
print(np.round(lh,4))
# np.round(lh,4)

[[-1.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.     0.    -0.    -1.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.     0.667 -0.    -0.667 -0.    -0.333 -0.    -0.   ]
 [-0.    -1.    -0.     0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.667 -0.    -0.333 -0.    -0.667 -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.     0.    -0.    -1.    -0.   ]
 [-0.    -0.    -0.333 -0.    -0.667 -0.     0.667 -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.    -1.    -0.     0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -1.   ]]


In [13]:
np.linalg.eigvalsh(lh)

array([-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.])

In [14]:
from random import randint

D = 3
seed = randint(0, 2<<32 - 1)
loss_mes = nsp.loss.MES(lh, [D, D], pout = False)
model = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
optm = RiemanNonTransUnitaryCG([[model, model]], [loss_mes], pout = False, lr = 0.01)
solver = UnitaryNonTransTs(optm, af=False)
ret = solver.run(3000, False)
print(f"res = {ret.fun} / seed = {seed}")
best_model = ret.model

target loss      : 2.0000000132
initial loss     : 2.6627191867
loss upper bound : 2.4880338572




  0%|          | 0/3000 [00:00<?, ?it/s]

No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 2.000000013242897 / seed = 3571751830


In [16]:
best_model[0].matrix()

tensor([[ 7.0711e-01, -5.0645e-01,  4.9347e-01],
        [-6.0105e-13, -6.9787e-01, -7.1622e-01],
        [ 7.0711e-01,  5.0645e-01, -4.9347e-01]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [18]:
R = np.array([
    [-1j/np.sqrt(2), 0, 1j/np.sqrt(2)],
    [1/np.sqrt(2), 0, 1/np.sqrt(2)],
    [0, 1j, 0],
])

In [19]:
UR = np.kron(R, R)

In [25]:
np.linalg.eigvalsh(stoquastic(UR @ lh @ UR.T))

array([-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.])

In [27]:
LHs = []
for act, loss_ in zip(optm.models_act, optm.loss_list):
    LHs.append(loss_._transform([best_model[s].matrix() for s in act], original = True).detach().numpy())

In [28]:
np.round(LHs[0],4)

array([[-0.333, -0.   , -0.   , -0.   , -0.667, -0.   , -0.   , -0.   ,
        -0.667],
       [-0.   ,  0.   ,  0.   , -1.   ,  0.   ,  0.   , -0.   ,  0.   ,
         0.   ],
       [-0.   ,  0.   ,  0.   , -0.   ,  0.   ,  0.   , -1.   ,  0.   ,
         0.   ],
       [-0.   , -1.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [-0.667,  0.   ,  0.   ,  0.   , -0.333, -0.   ,  0.   , -0.   ,
         0.667],
       [-0.   ,  0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   , -1.   ,
        -0.   ],
       [-0.   , -0.   , -1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [-0.   , -0.   ,  0.   ,  0.   , -0.   , -1.   ,  0.   ,  0.   ,
        -0.   ],
       [-0.667,  0.   ,  0.   ,  0.   ,  0.667, -0.   ,  0.   , -0.   ,
        -0.333]])

In [65]:
np.set_printoptions(precision=3)
print(np.round(LH,3))

[[-2. -0. -0. -0.  1. -0. -0. -0.  1.]
 [-0. -0. -0. -1.  0. -0. -0. -0.  0.]
 [-0. -0. -0. -0.  0. -0. -1. -0.  0.]
 [-0. -1. -0. -0.  0. -0. -0. -0.  0.]
 [ 1.  0.  0.  0. -2.  0.  0.  0. -1.]
 [-0. -0. -0. -0.  0.  0. -0. -1.  0.]
 [-0. -0. -1. -0.  0. -0. -0. -0.  0.]
 [-0. -0. -0. -0.  0. -1. -0.  0.  0.]
 [ 1.  0.  0.  0. -1.  0.  0.  0. -2.]]


In [66]:
E, V = np.linalg.eigh(LH)

In [207]:
Sz = np.zeros([3,3])
Sz[0,0] = 1
Sz[2,2] = -1
Sx = np.zeros([3, 3])
Sx[1,0] = Sx[0,1] = Sx[2,1] = Sx[1,2] = 1/np.sqrt(2)
Sy = np.zeros([3, 3], dtype=np.complex64)
Sy[1,0] = Sy[2,1] = 1j/np.sqrt(2)
Sy[0,1] = Sy[1,2] = -1j/np.sqrt(2)


SzSz = np.kron(Sz,Sz).astype(np.float64)
SxSx = np.kron(Sx,Sx).astype(np.float64)
SySy = np.kron(Sy,Sy).astype(np.float64)

h = SzSz + SxSx + SySy
J = [1, 10]
h = h + (h@h - np.eye(9))*J[1]
h = -h

/tmp/ipykernel_1596/2157545296.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  SySy = np.kron(Sy,Sy).astype(np.float64)


In [206]:
np.where(np.round(h,4) <= -1E-8)

(array([0, 1, 2, 2, 3, 4, 5, 6, 6, 7, 8]),
 array([0, 3, 2, 6, 1, 4, 7, 2, 6, 5, 8]))

In [85]:
u = best_model.matrix().detach().numpy()
U = np.kron(u, u)

In [208]:
np.set_printoptions(precision=3)
print(np.round(U.T @ h @ U
,4))

[[-1.e+01 -9.e-04 -1.e-04 -9.e-04  9.e+00 -0.e+00 -1.e-04 -0.e+00  9.e+00]
 [-9.e-04 -0.e+00 -0.e+00 -1.e+00  9.e-04 -0.e+00 -0.e+00 -0.e+00  9.e-04]
 [-1.e-04 -0.e+00 -0.e+00 -0.e+00  1.e-04 -0.e+00 -1.e+00 -0.e+00  1.e-04]
 [-9.e-04 -1.e+00 -0.e+00 -0.e+00  9.e-04 -0.e+00 -0.e+00 -0.e+00  9.e-04]
 [ 9.e+00  9.e-04  1.e-04  9.e-04 -1.e+01  0.e+00  1.e-04  0.e+00 -9.e+00]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00  0.e+00 -0.e+00 -1.e+00  0.e+00]
 [-1.e-04 -0.e+00 -1.e+00 -0.e+00  1.e-04 -0.e+00 -0.e+00 -0.e+00  1.e-04]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00 -1.e+00 -0.e+00  0.e+00  0.e+00]
 [ 9.e+00  9.e-04  1.e-04  9.e-04 -9.e+00  0.e+00  1.e-04  0.e+00 -1.e+01]]


In [211]:
h_ = U.T @ h @ U
# h_  = h
np.linalg.eigvalsh(stoquastic(h_))[-1], np.linalg.eigvalsh(h_)[-1]

(8.000000710002148, 1.000001132488213)

In [71]:
np.linalg.eigvalsh(h)

array([-1. , -1. , -1. , -1. , -1. ,  0.5,  1. ,  1. ,  1. ])

In [87]:
L = 4
H = sum_ham(lh, [[i, (i+1)%L] for i in range(L)], L, 3)

In [88]:
np.linalg.eigvalsh(H)

array([-2.66666667, -2.00000005, -2.00000005, -1.99999996, -1.19419191,
       -1.19419191, -1.19419191, -1.19419191, -1.1941919 , -1.1941919 ,
       -1.0971676 , -1.0971676 , -1.09716753, -1.09716753, -1.09716751,
        0.66666659,  0.66666659,  0.6666666 ,  0.66666669,  1.33333313,
        1.33333313,  1.33333317,  1.33333317,  1.33333317,  1.33333317,
        1.33333317,  1.33333317,  1.33333328,  1.33333328,  1.33333331,
        1.33333331,  1.33333331,  1.33333331,  1.33333333,  1.33333333,
        1.33333333,  1.33333335,  1.33333335,  1.33333339,  1.33333339,
        1.3333334 ,  1.86085844,  1.86085844,  1.86085851,  1.86085851,
        1.86085851,  1.86085851,  1.99999991,  2.43050073,  2.43050073,
        2.43050079,  2.43050079,  2.43050082,  2.66666657,  2.66666657,
        2.66666659,  2.66666659,  2.66666659,  3.33333313,  3.33333313,
        3.33333313,  3.33333313,  3.33333324,  3.33333324,  3.33333324,
        3.33333324,  3.3333333 ,  3.3333333 ,  3.3333333 ,  3.33